In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler

import torchvision
from torchvision import transforms, datasets

In [ ]:
!unzip catdogdata.zip

In [ ]:
data_dir= "kagglecatsanddogs_3367a/PetImages"

In [ ]:
image_size = 64

data_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.Grayscale(num_output_channels=1),
                                      transforms.ToTensor(),

                                     ])

dataset = datasets.ImageFolder(data_dir,transform = data_transforms)

classes = dataset.classes
print(classes)
img, label = dataset[0]
print(img.shape,label)

In [ ]:
img = img.reshape(64,64,-1)
plt.imshow(img,cmap="gray")

In [ ]:
batch_size = 128
valid_size = 0.20
test_size = 0.10

dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

split1 = int(np.floor(valid_size * dataset_size))
split2 = int(np.floor(test_size*dataset_size)) + split1
print(split1,split2)

valid_idx, test_idx, train_idx = indices[:split1], indices[split1:split2],indices[split2:]

training_data_sampler = SubsetRandomSampler(train_idx)
valid_data_sampler = SubsetRandomSampler(valid_idx)
test_data_sampler = SubsetRandomSampler(test_idx)

train_loader = DataLoader(dataset, sampler = training_data_sampler, batch_size = batch_size)
val_loader    = DataLoader(dataset, sampler = valid_data_sampler, batch_size = batch_size)
test_loader     = DataLoader(dataset, sampler = test_data_sampler, batch_size = batch_size)

In [3]:
class Model(nn.Module):
    def __init__(self):

        super(Model, self).__init__()
        self.network = nn.Sequential(

            # - x 1 x 64 x 64
            nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            # - x 32 x 32 x 32
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            # - x 64 x 16 x 16
            nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            # - x 128 x 8 x 8
            nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            # - x 256 x 4 x 4

            nn.Flatten(),
            nn.Linear(256*4*4,512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.network(x)
        return x


In [ ]:
device =  torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [4]:
model = Model().to(device)

In [ ]:
criterion = nn.BCELoss().to(device)
optimiser = optim.Adam(model.parameters(), lr= 0.001).to(device)

In [ ]:
def train(epochs):
  for epoch in range(epochs):
    model.train()
    for batch in train_loader:
      xb, labels = batch
      xb = xb.to(device)
      ypred = model(xb)
      labels = labels.to(device)
      loss = criterion(ypred,labels)
      loss.backward()
      optimiser.step()
      optimiser.zero_grad
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    val_size = 0
    for batch in val_loader:
      xb, labels = batch
      xb = xb.to(device)
      ypred = model(xb)
      labels = labels.to(device)
      loss = criterion(ypred,labels)
      val_loss += loss.data.item()*labels.shape[0]
      val_size += labels.shape[0]
      binary_preds = (ypred >= 0.5).int()
      correct_preds += (binary_preds == labels).sum().item()
    val_loss = val_loss/val_size
    val_accuracy = correct_preds/val_size
    print(f"Epoch: {epoch+1} val_loss= {val_loss:.4f} val_accuracy= {val_accuracy:.4f}")

In [ ]:
def test():
    model.eval()
    correct_preds = 0
    test_size = 0
    for batch in test_loader:
      xb, labels = batch
      xb = xb.to(device)
      ypred = model(xb)
      labels = labels.to(device)
      test_size += labels.shape[0]
      binary_preds = (ypred >= 0.5).int()
      correct_preds += (binary_preds == labels).sum().item()
    test_accuracy = correct_preds/test_size
    print(f"Accuracy on test data: {test_accuracy:.4f}")

In [ ]:
num_epochs = 20
train(num_epochs)

In [ ]:
test()